In [1]:
# Cell 1: Single File Download Test
import cdsapi
import os

# --- Configuration ---
# Create a dedicated folder for this test
output_dir = '../data/climate_raw/api_test/'
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, 'era5_land_usa_1981_01_temp.grib')

# --- API Request for a Single Month ---
print("Building request for January 1981...")
request_dictionary = {
    'format': 'grib',
    'variable': [
        '2m_temperature',
    ],
    'year': '1981',
    'month': '01',
    'day': [f'{d:02d}' for d in range(1, 32)],
    'time': [f'{h:02d}' for h in range(0, 24)],
    'area': [
        50, -105, 25, -80, # North, West, South, East
    ],
}

# --- Execute Download ---
try:
    if not os.path.exists(output_file):
        c = cdsapi.Client()
        print("Submitting API request to the CDS...")
        print(f"File will be saved to: {output_file}")
        
        c.retrieve(
            'reanalysis-era5-land',
            request_dictionary,
            output_file
        )
        print("\nDownload complete!")
        print(f"File size: {os.path.getsize(output_file) / 1e6:.2f} MB")
    else:
        print(f"File already exists, skipping download: {output_file}")

except Exception as e:
    print(f"\nAn error occurred: {e}")
    print("Please double-check that the 'climarisc_env' is active and your .cdsapirc file is correct.")

Building request for January 1981...


2025-09-03 19:21:21,011 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.


Submitting API request to the CDS...
File will be saved to: ../data/climate_raw/api_test/era5_land_usa_1981_01_temp.grib


2025-09-03 19:21:21,719 INFO Request ID is 1c1dd88f-e04f-49ca-acb3-b3f8419eb138
2025-09-03 19:21:21,813 INFO status has been updated to accepted
2025-09-03 19:21:33,878 INFO status has been updated to running
2025-09-03 19:23:20,026 INFO status has been updated to successful
                                                                                                                       


Download complete!
File size: 71.04 MB


In [2]:
# Cell 2: Load and Inspect the Downloaded GRIB Files
import xarray as xr

# --- File Paths ---
# Path to the file downloaded via our Python API script
api_file_path = r'../data/climate_raw/api_test/era5_land_usa_1981_01_temp.grib'

# Path to the file you downloaded manually from the website
manual_file_path = r'../data/climate_raw/api_test/data.grib'

# --- Test 1: Open the API-downloaded file ---
print("--- Attempting to open the API-downloaded file ---")
try:
    # We must explicitly use the 'cfgrib' engine to read GRIB files
    ds_api = xr.open_dataset(api_file_path, engine='cfgrib')
    print("SUCCESS: API file loaded correctly.")
    print(ds_api)
except Exception as e:
    print(f"FAILED to load API file. Error: {e}")

print("\n" + "="*50 + "\n")

# --- Test 2: Open the manually-downloaded file ---
print("--- Attempting to open the manually-downloaded file ---")
try:
    # Use the same engine to open the manual file
    ds_manual = xr.open_dataset(manual_file_path, engine='cfgrib')
    print("SUCCESS: Manually downloaded file loaded correctly.")
    print(ds_manual)
except Exception as e:
    print(f"FAILED to load manual file. Error: {e}")

--- Attempting to open the API-downloaded file ---


Can't create file '../data/climate_raw/api_test/era5_land_usa_1981_01_temp.grib.5b7b6.idx'
Traceback (most recent call last):
  File "C:\ProgramData\miniconda3\envs\climarisc\lib\site-packages\cfgrib\messages.py", line 274, in itervalues
    yield self.filestream.message_from_file(file, errors=errors)
  File "C:\ProgramData\miniconda3\envs\climarisc\lib\site-packages\cfgrib\messages.py", line 341, in message_from_file
    return Message.from_file(file, offset, **kwargs)
  File "C:\ProgramData\miniconda3\envs\climarisc\lib\site-packages\cfgrib\messages.py", line 105, in from_file
    raise EOFError("End of file: %r" % file)
EOFError: End of file: <_io.BufferedReader name='../data/climate_raw/api_test/era5_land_usa_1981_01_temp.grib'>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\miniconda3\envs\climarisc\lib\site-packages\cfgrib\messages.py", line 539, in from_indexpath_or_filestream
    self = cls.from_fi

FAILED to load API file. Error: No valid message found: '../data/climate_raw/api_test/era5_land_usa_1981_01_temp.grib'


--- Attempting to open the manually-downloaded file ---
SUCCESS: Manually downloaded file loaded correctly.
<xarray.Dataset> Size: 194MB
Dimensions:     (time: 32, step: 24, latitude: 251, longitude: 251)
Coordinates:
    number      int64 8B ...
  * time        (time) datetime64[ns] 256B 1980-12-31 1981-01-01 ... 1981-01-31
  * step        (step) timedelta64[ns] 192B 01:00:00 ... 1 days 00:00:00
    surface     float64 8B ...
  * latitude    (latitude) float64 2kB 50.0 49.9 49.8 49.7 ... 25.2 25.1 25.0
  * longitude   (longitude) float64 2kB -105.0 -104.9 -104.8 ... -80.1 -80.0
    valid_time  (time, step) datetime64[ns] 6kB ...
Data variables:
    t2m         (time, step, latitude, longitude) float32 194MB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts


C:\ProgramData\miniconda3\envs\climarisc\lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(


In [3]:
# Cell 1: Single File Download Test (Final Robust Version)
import cdsapi
import os
import xarray as xr

# --- Configuration ---
output_dir = '../data/climate_raw/api_test/'
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, 'era5_land_usa_1981_01_temp_robust.grib')

# --- API Request for a Single Month ---
print("Building ROBUST request for January 1981...")
request_dictionary = {
    'format': 'grib',
    'variable': [
        '2m_temperature',
    ],
    'year': '1981',
    'month': '01',
    'day': [f'{d:02d}' for d in range(1, 32)],
    'time': [f'{h:02d}:00' for h in range(0, 24)],
    'area': [
        50, -105, 25, -80, # North, West, South, East
    ],
    # ADDED: This is a common option to ensure the file is properly packaged
    'download_format': 'unarchived'
}

# --- Execute Download ---
try:
    if not os.path.exists(output_file):
        c = cdsapi.Client()
        print("Submitting API request to the CDS...")
        print(f"File will be saved to: {output_file}")
        
        c.retrieve(
            'reanalysis-era5-land',
            request_dictionary,
            output_file
        )
        print("\nDownload complete!")
    else:
        print(f"File already exists, skipping download: {output_file}")

    # --- Verification Step ---
    print("\n--- Verifying the downloaded file ---")
    ds_test = xr.open_dataset(output_file, engine='cfgrib')
    print("SUCCESS! The downloaded file was opened correctly.")
    print(ds_test)

except Exception as e:
    print(f"\nAn error occurred: {e}")

Building ROBUST request for January 1981...


2025-09-03 19:37:43,452 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.


Submitting API request to the CDS...
File will be saved to: ../data/climate_raw/api_test/era5_land_usa_1981_01_temp_robust.grib


2025-09-03 19:37:43,684 INFO Request ID is 45dc2172-6a79-436b-a265-4d5732e7ec64
2025-09-03 19:37:43,867 INFO status has been updated to accepted
2025-09-03 19:37:57,476 INFO status has been updated to running
2025-09-03 19:40:35,990 INFO status has been updated to successful
                                                                                                                       


Download complete!

--- Verifying the downloaded file ---
SUCCESS! The downloaded file was opened correctly.
<xarray.Dataset> Size: 194MB
Dimensions:     (time: 32, step: 24, latitude: 251, longitude: 251)
Coordinates:
    number      int64 8B ...
  * time        (time) datetime64[ns] 256B 1980-12-31 1981-01-01 ... 1981-01-31
  * step        (step) timedelta64[ns] 192B 01:00:00 ... 1 days 00:00:00
    surface     float64 8B ...
  * latitude    (latitude) float64 2kB 50.0 49.9 49.8 49.7 ... 25.2 25.1 25.0
  * longitude   (longitude) float64 2kB -105.0 -104.9 -104.8 ... -80.1 -80.0
    valid_time  (time, step) datetime64[ns] 6kB ...
Data variables:
    t2m         (time, step, latitude, longitude) float32 194MB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Med

C:\ProgramData\miniconda3\envs\climarisc\lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
